<a href="https://colab.research.google.com/github/Dibya069/Generative_AI-Codes-to-learn-/blob/main/TeleBot_with_audioAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
2+2

4

## LLAMA-3 model for Chat bot

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


## Tele Gram Bot

In [ ]:
!pip install python-dotenv
!pip install --force-reinstall -v "aiogram==2.23.1"

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
  Obtaining dependency information for aiogram==2.23.1 from https://files.pythonhosted.org/packages/b1/44/adf2dddedb324cdc856ff77512e5dfc7e70c2d244c4efdc30e4836e8e88a/aiogram-2.23.1-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp<3.9.0,>=3.8.0 from https://files.pythonhosted.org/packages/41/8e/4c48881316bbced3d13089c4d0df4be321ce79a0c695d82dee9996aaf56b/aiohttp-3.8.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for Babel<2.10.0,>=2.9.1 from https://files.pythonhosted.org/packages/aa/96/4ba93c5f40459dc850d25f9ba93f869a623e77aaecc7a9344e19c01942cf/Babel-2.9.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2021.10.8 from https://files.pythonhosted.org/packages/12/90/3c9ff0512038035f59d279fddeb79f5f1eccd8859f06d6163c58798b9487/certifi-2024.8.30-py3-none-any.whl.metadata
  Using cached certifi-2024.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 25.1 MB/s eta 0:00:00


In [ ]:
%%writefile tele_audio_bot.py

import logging
from aiogram import Bot, Dispatcher, executor, types
from dotenv import load_dotenv
import os
from groq import Groq
import google.generativeai as genai
from google.colab import userdata

load_dotenv()
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
client = Groq(api_key=userdata.get('GROQ_API_KEY'))

import requests

DEEPGRAM_URL = "https://api.deepgram.com/v1/speak?model=aura-asteria-en"
DEEPGRAM_API_KEY = userdata.get('DEEPGRAM_API_KEY')

class SpeakerAnalysis:
    def __init__(self, audio_file_path, pro1, pro2):
        self.audio_file = self.upload_audio_file(audio_file_path)
        self.pro_1 = pro1
        self.pro_2 = pro2

    def upload_audio_file(self, path):
        try:
            audio_file = genai.upload_file(path=path)
            return audio_file
        except Exception as e:
            print(f"Error uploading file: {e}")
            return None

    def D_response(self):
        if not self.audio_file:
            return "Error: No audio file uploaded"

        model_config = {"temperature": 0.1}
        model = genai.GenerativeModel('gemini-1.5-flash-latest', generation_config=model_config)
        response = model.generate_content([self.audio_file, self.pro_1])
        return response.text

    def A_response(self):
        if not self.audio_file:
            return "Error: No audio file uploaded"

        model_config = {"temperature": 0.3}
        model = genai.GenerativeModel('gemini-1.5-flash-latest', generation_config=model_config)
        response = model.generate_content([self.audio_file, self.pro_2])
        return response.text


pro1 = """
  Do the full Transcription with all the necessary punctuation of this audio.
"""

pro2 = f"""
  Give the full analysis and summary of this audio file.
"""

def TTS(payload):
    headers = {
        "Authorization": f"Token {DEEPGRAM_API_KEY}",
        "Content-Type": "application/json"
    }

    audio_file_path = "audio.wav"  # Path to save the audio file

    with open(audio_file_path, 'wb') as file_stream:
        response = requests.post(DEEPGRAM_URL, headers=headers, json=payload, stream=True)
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file_stream.write(chunk)  # Write each chunk of audio data to the file

    print("Audio download complete")

class Reference:
    '''
    A class to store previously response from the chartGPT API
    '''
    def __init__(self) -> None:
        self.response = ""

reference = Reference()
TOKEN = userdata.get('TELE_TOKEN')

# Initialize bot and dispatcher
bot = Bot(token = TOKEN)
dispatcher = Dispatcher(bot)

def clear_past():
    '''
    A function to clear the previous conversation
    '''
    reference.response = ""


@dispatcher.message_handler(commands=['start'])
async def command_start_handler(message: types.Message):
    """
    This handler receives messages with `/start` or `/help `command
    """
    await message.reply("Hi\nI am tele Bot!\n Created By Dibya")


@dispatcher.message_handler(commands=['help'])
async def command_start_handler(message: types.Message):
    """
    This handler displays the help menu.
    """
    help_com = """
    Hi there, do you need some help!!!
    Refer to the following options:
    /start - to start the conversation
    /clear - to remove the previous conversation
    /help - how did you get this, you silly!
    Hope this helps :)
    """
    await message.reply(help_com)


@dispatcher.message_handler(commands=['clear'])
async def command_clear_handler(message: types.Message):
    """
    A handler to clear the previous conversation and context
    """
    clear_past()
    await message.reply("Cleared all your previous conversations.")


async def chatgpt(message: types.Message, query=None):
    """
    A handler to process the user's input and generate a response using the ChatGPT AI
    """
    user_query = query if query else message.text
    print(f">>> USER: \n\t{user_query}")

    response = client.chat.completions.create(
        messages=[
            {"role": "assistant", "content": reference.response},  # previous assistant response
            {"role": "user", "content": user_query}  # query or user input
        ],
        model="llama-3.1-8b-instant",
        max_tokens=150
    )
    reference.response = response.choices[0].message.content
    await message.reply(f"Response: {reference.response}")

    if reference.response:
        audio_path = "/content/audio.wav"
        payload = {"text" : reference.response}
        try:
            print("Generating audio with TTS...")
            TTS(payload=payload)
            print("Audio generation complete.")

            audio = types.InputFile(os.path.expanduser(audio_path))
            await message.reply_audio(audio, title = "Response")
        except Exception as e:
            logging.error(f"Error generating audio: {e}")
            await message.reply("An error occurred while generating the audio response.")
            return # Stop further processing if audio generation fails


async def download_audio_file(file_id, bot):
    file_info = await bot.get_file(file_id)
    file_path = file_info.file_path
    downloaded_file = await bot.download_file(file_path)
    download_dir = "downloads"

    if not os.path.exists(download_dir):
        os.makedirs(download_dir)

    file_name = f"{download_dir}/{file_id}.ogg"
    with open(file_name, 'wb') as f:
        f.write(downloaded_file.read())
    return file_name


@dispatcher.message_handler(content_types=['voice', 'audio'])
async def handle_audio(message: types.Message):
    file_id = message.voice.file_id if message.content_type == 'voice' else message.audio.file_id
    print(f"Received audio file with ID: {file_id}")
    await bot.send_message(chat_id=message.chat.id, text=f"Received audio file with ID: {file_id}")

    audio_file_path = await download_audio_file(file_id, bot)

    await bot.send_message(chat_id=message.chat.id, text="Please wait a moment, the analysis is in process...")

    # Create SpeakerAnalysis instance
    gen = SpeakerAnalysis(audio_file_path, pro1, pro2)
    res1 = gen.D_response()
    res2 = gen.A_response()

    # Send an inline keyboard with options
    keyboard = types.InlineKeyboardMarkup(row_width=3)
    transcription_btn = types.InlineKeyboardButton("Transcription", callback_data="transcription")
    analysis_btn = types.InlineKeyboardButton("Analysis", callback_data="analysis")
    query_btn = types.InlineKeyboardButton("Query", callback_data="query")
    keyboard.add(transcription_btn, analysis_btn, query_btn)

    await bot.send_message(chat_id=message.chat.id, text="Choose an option:", reply_markup=keyboard)

    # Store the results in a global or class variable to use in callback queries
    bot.audio_results = {"res1": res1, "res2": res2}


@dispatcher.callback_query_handler(lambda callback_query: True)
async def callback_inline_handler(callback_query: types.CallbackQuery):
    """
    Handle the button clicks for transcription, analysis, and query
    """
    data = callback_query.data

    # Retrieve the stored audio results
    res1 = bot.audio_results["res1"]
    res2 = bot.audio_results["res2"]

    message = callback_query.message

    if data == "transcription":
        await bot.send_message(callback_query.from_user.id, text=res1)

    elif data == "analysis":
        await bot.send_message(callback_query.from_user.id, text=res2)

    elif data == "query":
        await chatgpt(message, query=res1)

    await bot.answer_callback_query(callback_query.id)  # Acknowledge the callback query


if __name__ == "__main__":
    executor.start_polling(dispatcher, skip_updates=False)


Overwriting tele_audio_bot.py


In [ ]:
!python /content/tele_audio_bot.py

Received audio file with ID: AwACAgUAAxkBAAIBhGcHdYrcTOGEiNl_1OMflCBWg3YOAAJoEwAC_s04VGugwqa_T_JQNgQ
>>> USER: 
	What type of techniques used in fine-tuning and uh what are these techniques can you tell me? 

Generating audio with TTS...
Audio download complete
Audio generation complete.
Goodbye!
